In [2]:
from dotenv import load_dotenv
import os
from pydantic import BaseModel, Field, model_validator
def load_api_keys():
    """
    Load multiple LLM API keys at once from .env file
    Returns a dictionary of all loaded keys
    """
    # Load environment variables from .env
    load_dotenv()
    
    # Define all required API keys
    required_keys = [
        "OPENAI_API_KEY",
        "OPENAI_API_KEY_rise",
        "AZURE_OPENAI_API_KEY",
        "ANTHROPIC_API_KEY",
        "GOOGLE_GEMINI_KEY",
        "Grok_API_KEY"
    ]
    
    # Load all keys at once using dictionary comprehension
    api_keys = {key: os.getenv(key) for key in required_keys}
    
    # Check for missing keys
    missing_keys = [key for key, value in api_keys.items() if not value]
    if missing_keys:
        raise ValueError(f"Missing required API keys: {', '.join(missing_keys)}")
        
    return api_keys

# Example usage
try:
    keys = load_api_keys()
    
    # Unpack all keys at once
    openai_key, azure_key, anthropic_key, xai_key, gemini_key = (
        keys["OPENAI_API_KEY"],
        keys["AZURE_OPENAI_API_KEY"],
        keys["ANTHROPIC_API_KEY"],
        keys["Grok_API_KEY"],
        keys["GOOGLE_GEMINI_KEY"]
    )
    
    print("All API keys loaded successfully!")
    
except Exception as e:
    print(f"Error: {e}")


class Base_Parser(BaseModel):
    Answer: str = Field(description="Just the answer only to the provided question without explanation")
    Confidence: str = Field(description="Confidence for final answer; it should be a floating number from 0 to 1 where 0 to 0.5 means less confident and 0.5 to 1 means more confident")



All API keys loaded successfully!


In [3]:
import pandas as pd
from langchain_google_genai import ChatGoogleGenerativeAI
import os

In [69]:
df = pd.read_csv('data/final_df_sample.csv')

In [70]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
# Define different prompt formats
messages_format_no_exp_with_conf = [
    ("system", "You are a helpful and concise assistant. At the end of your response, include only the answer (without explanation) clearly marked as Final Answer on a new line as well as your confidence (from 0 to 1) of your answer. Example: Question: What is 2 + 2? Final Answer: 4, Confidence: 1"),
    ("human", df['Question'].iloc[1000])
]

messages_format_with_exp_no_conf = [
    ("system", "You are a helpful and concise assistant. Provide a detailed explanation of your reasoning, then include only the answer clearly marked as Final Answer on a new line. Example: Question: What is 2 + 2? Here's how we solve this: We add the numbers 2 and 2 together which equals 4. Final Answer: 4"),
    ("human", df['Question'].iloc[1000])
]

messages_format_no_exp_no_conf = [
    ("system", "You are a helpful and concise assistant. Provide only the answer without explanation. Example: Question: What is 2 + 2? Answer: 4"),
    ("human", df['Question'].iloc[1000])
]

messages_format_with_exp_with_conf = [
    ("system", "You are a helpful and concise assistant. Provide a detailed explanation of your reasoning, then include the answer clearly marked as Final Answer on a new line along with your confidence (0 to 1). Example: Question: What is 2 + 2? Here's how we solve this: We add the numbers 2 and 2 together which equals 4. Final Answer: 4, Confidence: 1"),
    ("human", df['Question'].iloc[1000])
]

parser = JsonOutputParser(pydantic_object=Base_Parser)
chat_prompt = ChatPromptTemplate(messages_format_no_exp_with_conf, partial_variables={"format_instructions": parser.get_format_instructions()})

In [71]:
# Don't run this code; Only works on Guangya's computer due to credentials
model = 'gemini-2.0-flash-thinking-exp'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '/home/guangya/.config/gcloud/application_default_credentials.json'

llm_gemini = ChatGoogleGenerativeAI(model=model)

In [25]:
message_gemini_1 = llm_gemini.invoke(messages_format_no_exp_with_conf)
message_gemini_2 = llm_gemini.invoke(messages_format_with_exp_no_conf)
message_gemini_3 = llm_gemini.invoke(messages_format_no_exp_no_conf)
message_gemini_4 = llm_gemini.invoke(messages_format_with_exp_with_conf)

In [26]:
message_gemini_1 

AIMessage(content='Final Answer: B) False, Confidence: 5', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-fbb32420-fe74-434b-9d26-5bb60134a933-0', usage_metadata={'input_tokens': 231, 'output_tokens': 11, 'total_tokens': 242, 'input_token_details': {'cache_read': 0}})

In [27]:
message_gemini_2

AIMessage(content='Here\'s how we determine if the statement "Max is sour" is true or false based on the provided context:\n\n1. **Max is a yumpus.** (Given in the last sentence)\n2. **Each yumpus is a dumpus.** (From the statement "Each yumpus is a dumpus.")\n   Therefore, Max is a dumpus.\n3. **Dumpuses are wumpuses.** (From the statement "Dumpuses are wumpuses.")\n   Therefore, Max is a wumpus.\n4. **Every wumpus is an impus.** (From the statement "Every wumpus is an impus.")\n   Therefore, Max is an impus.\n5. **Impuses are tumpuses.** (From the statement "Impuses are tumpuses.")\n   Therefore, Max is a tumpus.\n6. **Tumpuses are not sour.** (From the statement "Tumpuses are not sour.")\n   Therefore, Max is not sour.\n\nBased on this chain of reasoning, the statement "Max is sour" is false.\n\nFinal Answer: B) False', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-7334e9

In [28]:
message_gemini_3

AIMessage(content='B) False', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-f9092090-55f8-4a90-a5b7-32ead8588b2f-0', usage_metadata={'input_tokens': 191, 'output_tokens': 3, 'total_tokens': 194, 'input_token_details': {'cache_read': 0}})

In [29]:
message_gemini_4

AIMessage(content='Here\'s how we solve this:\n\nWe are given a series of statements and asked to determine if "Max is sour" is true or false. Let\'s trace the properties of Max based on the given information:\n\n1.  **Max is a yumpus.** (Given: "Max is a yumpus.")\n2.  **Each yumpus is a dumpus.** (Given: "Each yumpus is a dumpus.")\n    *   Therefore, Max is a dumpus.\n3.  **Dumpuses are wumpuses.** (Given: "Dumpuses are wumpuses.")\n    *   Therefore, Max is a wumpus.\n4.  **Every wumpus is an impus.** (Given: "Every wumpus is an impus.")\n    *   Therefore, Max is an impus.\n5.  **Impuses are tumpuses.** (Given: "Impuses are tumpuses.")\n    *   Therefore, Max is a tumpus.\n6.  **Tumpuses are not sour.** (Given: "Tumpuses are not sour.")\n    *   Therefore, Max is not sour.\n\nSince Max is a tumpus, and tumpuses are not sour, Max is not sour.  The statement "Max is sour" is therefore false.\n\nRationale: We followed a chain of logical deductions starting from the fact that Max is a

In [41]:
chat_prompt = ChatPromptTemplate(messages_format_no_exp_with_conf)
chain_gemini = chat_prompt | llm_gemini 
message = chain_gemini.invoke({})

In [42]:
chat_prompt

ChatPromptTemplate(input_variables=[], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful and concise assistant. At the end of your response, include only the answer (without explanation) clearly marked as Final Answer on a new line as well as your confidence (from 0 to 1) of your answer. Example: Question: What is 2 + 2? Final Answer: 4, Confidence: 1'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="Context: Jompuses are not shy. Jompuses are yumpuses. Each yumpus is aggressive. Each yumpus is a dumpus. Dumpuses are not wooden. Dumpuses are wumpuses. Wumpuses are red. Every wumpus is an impus. Each impus is opaque. Impuses are tumpuses. Numpuses are sour. Tumpuses are not sour. Tumpuses are vumpuses. Vumpuses are earthy. Every vumpus is a zumpus. Zumpuses are sm

In [43]:
message.content

'Final Answer: B) False, Confidence: 1'

In [31]:
message.content

'Final Answer: B) False, Confidence: 5'

In [72]:
import pandas as pd
from langchain_xai import ChatXAI
import os

llm_xai = ChatXAI(model = "grok-2-1212",xai_api_key = xai_key,logprobs=1)

# llm_xai.invoke(messages_format_with_exp_with_conf)

In [75]:
def process_with_xai(row, row_index):
    print(f"Processing row {row_index}...")  # Add logging to track progress
    
    try:
        # Create messages format for this specific question
        messages = [
            ("system", "You are a helpful assistant. Please solve problems step by step:\n1. First, carefully read and understand the question\n2. Break down the key components and information given\n3. Explain your reasoning process clearly\n4. Show your work and calculations if applicable\n5. At the end of your response, include only the answer clearly marked as 'Final Answer' on a new line as well as your confidence (from 0 to 1) of your answer.\n\nExample:\nQuestion: What is 2 + 2?\nLet me solve this step by step:\n1. We are adding two numbers: 2 and 2\n2. Basic addition tells us 2 + 2 = 4\n3. This is a simple arithmetic fact that I am completely confident about\nFinal Answer: 4, Confidence: 1"),
            ("human", f"Please solve this question step by step:\n{row['Question']}")
        ]
        
        # Get response from XAI
        response = llm_xai.invoke(messages)
        
        # Extract metadata
        metadata = {
            'token_usage': response.response_metadata['token_usage'],
            'logprobs': response.response_metadata['logprobs']
        }
        
        # Extract the final answer and confidence from the response
        content = response.content
        
        # Simple parsing
        if 'Final Answer:' in content:
            final_answer = content.split('Final Answer:')[1].split(',')[0].strip()
            confidence = content.split('Confidence:')[1].strip() if 'Confidence:' in content else '0.5'
        else:
            final_answer = content.strip()
            confidence = '0.5'
            
        return pd.Series({
            'xai_answer': final_answer,
            'xai_confidence': confidence,
            'xai_full_response': content,
            'token_usage': str(metadata['token_usage']),
            'logprobs': str(metadata['logprobs'])
        })
    except Exception as e:
        print(f"Error processing row {row_index}: {str(e)}")  # Add error logging
        return pd.Series({
            'xai_answer': f'Error: {str(e)}',
            'xai_confidence': '0',
            'xai_full_response': '',
            'token_usage': '',
            'logprobs': ''
        })

# Test on first 100 samples
sample_size = 1000
test_df_mc = df.sample(n=sample_size).copy().reset_index(drop=True)

# Process each row with progress bar
from tqdm import tqdm
results = []
for index, row in tqdm(test_df_mc.iterrows(), total=len(test_df_mc), desc="Processing sample rows"):
    result = process_with_xai(row, index)
    results.append(result)

# Update the test dataframe with results
for col in ['xai_answer', 'xai_confidence', 'xai_full_response', 'token_usage', 'logprobs']:
    test_df_mc[col] = [result[col] for result in results]

# Save the sample results
test_df_mc.to_csv('xai_results_1200_samples.csv', index=False)
print("Sample processing complete. Results saved to 'xai_results_1200_samples.csv'")

# Print some basic statistics
print("\nProcessing Statistics:")
print(f"Total rows processed: {len(test_df_mc)}")
print(f"Successful responses: {len(test_df_mc[test_df_mc['xai_answer'].str.startswith('Error:') == False])}")
print(f"Failed responses: {len(test_df_mc[test_df_mc['xai_answer'].str.startswith('Error:')])}")

Processing sample rows:   0%|          | 0/1000 [00:00<?, ?it/s]

Processing row 0...


Processing sample rows:   0%|          | 1/1000 [00:04<1:19:36,  4.78s/it]

Processing row 1...


Processing sample rows:   0%|          | 2/1000 [00:17<2:39:07,  9.57s/it]

...

Processing row 998...


Processing sample rows: 100%|█████████▉| 999/1000 [3:26:29<00:11, 11.68s/it]

Processing row 999...


Processing sample rows: 100%|██████████| 1000/1000 [3:26:35<00:00, 12.40s/it]


Sample processing complete. Results saved to 'xai_results_1200_samples.csv'

Processing Statistics:
Total rows processed: 1000
Successful responses: 996
Failed responses: 4


In [78]:
test_df_mc['xai_confidence'].value_counts()

xai_confidence
1                                                                                                                                                                                                                                                                                                       797
0.9                                                                                                                                                                                                                                                                                                      87
0.95                                                                                                                                                                                                                                                                                                     79
0.8                                                                                  

In [53]:
test_df

,Question,Correct Answer,Name,Category,Type,xai_answer,xai_confidence,xai_full_response,token_usage,logprobs
0,"Today, David went to the clothing store. Betwe...",C,BigBench_hard,temporal_sequences,Logical,(C) 6am to 8am,1,David could have gone to the clothing store be...,"{'completion_tokens': 59, 'prompt_tokens': 259...","{'content': [{'token': ' David', 'bytes': [32,..."
1,"Today, Ashley went to the bookstore. Between w...",D,BigBench_hard,temporal_sequences,Logical,(D) 7am to 11am,1,Ashley could have gone to the bookstore betwee...,"{'completion_tokens': 60, 'prompt_tokens': 247...","{'content': [{'token': ' Ashley', 'bytes': [32..."
2,"Today, Samantha went to the bakery. Between wh...",B,BigBench_hard,temporal_sequences,Logical,B,1,Samantha could have gone to the bakery between...,"{'completion_tokens': 51, 'prompt_tokens': 263...","{'content': [{'token': ' Samantha', 'bytes': [..."
3,"Today, Thomas went to the gas station. Between...",A,BigBench_hard,temporal_sequences,Logical,(A) 8am to 9am,1,Thomas could have gone to the gas station betw...,"{'completion_tokens': 59, 'prompt_tokens': 259...","{'content': [{'token': ' Thomas', 'bytes': [32..."
4,"Today, Anthony went to the park. Between what ...",D,BigBench_hard,temporal_sequences,Logical,(D) 8pm to 9pm,1,Anthony could have gone to the park between 8p...,"{'completion_tokens': 54, 'prompt_tokens': 216...","{'content': [{'token': ' Anthony', 'bytes': [3..."
...,...,...,...,...,...,...,...,...,...,...
2995,What is the domain of the function $f(x) = \fr...,"(2,12) \cup (12,102)",MATH-500,4,Math,\((2,1,The domain of the function \( f(x) = \frac{2-x...,"{'completion_tokens': 305, 'prompt_tokens': 12...","{'content': [{'token': ' The', 'bytes': [32, 8..."
2996,Let $z = 1+i$ and $w = \dfrac{3z+1}{5z+7}$. Fi...,\frac{5}{13},MATH-500,3,Math,$\frac{5}{13}$,1,To find $|w|$ where $z = 1+i$ and $w = \frac{3...,"{'completion_tokens': 667, 'prompt_tokens': 10...","{'content': [{'token': ' To', 'bytes': [32, 84..."
2997,An equiangular octagon has four sides of lengt...,\frac{7}{2},MATH-500,5,Math,\(\frac{3}{2}\),1,"To find the area of the equiangular octagon, w...","{'completion_tokens': 507, 'prompt_tokens': 12...","{'content': [{'token': ' To', 'bytes': [32, 84..."
2998,A sequence $(a_n)$ is defined as follows:\n\[a...,-1,MATH-500,5,Math,-1,1,Given the sequence \(a_{i+1} = \frac{1}{1 - a_...,"{'completion_tokens': 1795, 'prompt_tokens': 1...","{'content': [{'token': ' Given', 'bytes': [32,..."


In [65]:
test_df_mc = test_df[(test_df['Name'].str.contains('BigBench')) | (test_df['Name'].str.contains('ProntoQA'))]

In [68]:
test_df_mc[test_df_mc['Name'] == 'ProntoQA']['xai_answer'].iloc[-1]

'Error: Error code: 429 - {\'code\': \'Some resource has been exhausted\', \'error\': \'Too many requests: RejectLimits(LimitsInfo { id: Buf("/team:80bc4b9b-a776-47b6-b525-3de82011fc6c/u:59f182a3-a0f4-4155-b059-732f98e3ff08"), req_type: Buf("/rt:grok-2-1212"), actual: Values { rps: 0, rph: 1201 }, expected: Values { rps: 1, rph: 1200 } })\'}'

In [59]:
test_df['Name'].value_counts()

Name
BigBench_hard    500
BigBench_easy    500
ProntoQA         500
GSM8K            500
GSM8K_hard       500
MATH-500         500
Name: count, dtype: int64

In [48]:
message.response_metadata['token_usage']

{'completion_tokens': 42,
 'prompt_tokens': 238,
 'total_tokens': 280,
 'completion_tokens_details': None,
 'prompt_tokens_details': {'audio_tokens': 0,
  'cached_tokens': 0,
  'text_tokens': 238,
  'image_tokens': 0}}

In [49]:
message.response_metadata['logprobs']

{'content': [{'token': ' Max',
   'bytes': [32, 77, 97, 120],
   'logprob': 0.0,
   'top_logprobs': []},
  {'token': ' is',
   'bytes': [32, 105, 115],
   'logprob': 0.0,
   'top_logprobs': []},
  {'token': ' a', 'bytes': [32, 97], 'logprob': 0.0, 'top_logprobs': []},
  {'token': ' y', 'bytes': [32, 121], 'logprob': 0.0, 'top_logprobs': []},
  {'token': 'ump',
   'bytes': [117, 109, 112],
   'logprob': 0.0,
   'top_logprobs': []},
  {'token': 'us', 'bytes': [117, 115], 'logprob': 0.0, 'top_logprobs': []},
  {'token': ',', 'bytes': [44], 'logprob': 0.0, 'top_logprobs': []},
  {'token': ' and',
   'bytes': [32, 97, 110, 100],
   'logprob': 0.0,
   'top_logprobs': []},
  {'token': ' y',
   'bytes': [32, 121],
   'logprob': -0.10077041,
   'top_logprobs': []},
  {'token': 'ump',
   'bytes': [117, 109, 112],
   'logprob': 0.0,
   'top_logprobs': []},
  {'token': 'uses',
   'bytes': [117, 115, 101, 115],
   'logprob': 0.0,
   'top_logprobs': []},
  {'token': ' are',
   'bytes': [32, 97, 114,

In [35]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

llm_openai = ChatOpenAI(model="gpt-4o-mini",temperature=1)

message_openai = llm_openai.invoke(messages_format_with_exp_with_conf)

In [36]:
message_openai.content

'To determine whether the statement "Max is sour" is true or false, let\'s analyze the information given in the context step by step.\n\n1. **Max is a yumpus:** \n   - From the context, we know that all yumpuses are aggressive and Jompuses are yumpuses. Therefore, Max is not shy and is aggressive.\n\n2. **All yumpuses are dumpuses:** \n   - Since Max is a yumpus, he must also be a dumpus.\n\n3. **Dumpuses are not wooden; they are wumpuses:** \n   - This means that Max, being a dumpus, is not wooden and is also a wumpus.\n\n4. **Wumpuses are red and every wumpus is an impus:** \n   - Max, as a wumpus, is red and is also classified as an impus.\n\n5. **Impuses are opaque, and impuses are tumpuses:** \n   - Max being an impus means he is also a tumpus.\n\n6. **Numpuses are sour but tumpuses are not sour:** \n   - Since Max is a tumpus, he cannot be sour.\n\nGiven this reasoning, since Max is classified as a tumpus and tumpuses are definitively stated to not be sour, the statement "Max is 

In [37]:
# import os

# os.environ["AZURE_OPENAI_ENDPOINT"] = "https://rtp2-gpt35.openai.azure.com/"
# os.environ["AZURE_OPENAI_API_VERSION"] = "2024-08-06"
# os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = "gpt-4o"

# from langchain_core.messages import HumanMessage
# from langchain_openai import AzureChatOpenAI

# llm_azure = AzureChatOpenAI(
#     openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
#     azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
# )

In [38]:
from langchain_anthropic import ChatAnthropic
llm_claude = ChatAnthropic(
    model="claude-3-5-haiku-20241022")

In [40]:
llm_claude.invoke(messages_format_with_exp_with_conf)

AIMessage(content="Let's solve this step by step:\n\n1. We know Max is a yumpus from the context.\n2. All yumpuses are dumpuses. \n3. Dumpuses are wumpuses. \n4. Wumpuses are impuses. \n5. Impuses are tumpuses.\n6. Tumpuses are vumpuses.\n7. Vumpuses are zumpuses.\n\nNow let's check the characteristic of tumpuses:\n8. Tumpuses are not sour.\n9. Tumpuses are vumpuses.\n10. Vumpuses are zumpuses.\n11. Zumpuses are small and rompuses.\n\nBut the crucial point is: Tumpuses are not sour.\n\nSince Max is a tumpus (through the chain of relationships), Max is not sour.\n\nFinal Answer: B) False, Confidence: 1", additional_kwargs={}, response_metadata={'id': 'msg_01XA7u9ip6pMrALzcdsouKzu', 'model': 'claude-3-5-haiku-20241022', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 278, 'output_tokens': 211, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0}}, id='run-9d42a1ea-ac0d-4d5e-910e-57f343d14dd2-0', usage_metadata={'input_tokens': 278, 'output_tokens': 

In [19]:
from langchain.prompts.chat import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
import json

In [20]:
# df = pd.read_csv('data/GSM8K.csv')

In [14]:
row = df.iloc[0]
subject = row['category']
reasoning_type = row['Type']
dataset = row['name']
question = row['Question']
prev_answer = row['parsed_answer']
rationale = row['reasoning']

arguments_dict = {
    'question': question,
    'rationale': rationale
}

In [45]:
refiner_json_link = 'prompts/refiner.json'
with open(refiner_json_link) as f:
    refiner_message = []
    for key,val in json.load(f).items():
        if key == 'system':
            val += '\n{format_instructions}'
        refiner_message.append((key,val))
feedback_json_link = 'prompts/feedback.json'
with open(feedback_json_link) as f:
    feedback_message = []
    for key,val in json.load(f).items():
        if key == 'system':
            val += '\n{format_instructions}'
        feedback_message.append((key,val))

In [16]:
refiner_message

[('system',
  'Using the provided question, rationale, and feedback, refine the rationale by addressing the issues raised. Implement the suggestions from the feedback to improve clarity, correctness, and consistency. Provide the revised rationale, the updated answer, and indicate whether further refinement is necessary.\n{format_instructions}'),
 ('human',
  'Question: {question}\nRationale: {rationale}\nFeedback: {feedback}')]

In [47]:
from pydantic import BaseModel, Field, model_validator
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate


class RefinerParser(BaseModel):
    answer: str = Field(description="The final answer of the given question. ")
    revised_rationale: str = Field(description="The new step-by-step reasoning.")
    


class FeedbackParser(BaseModel):
    feedback: str = Field(description="Specific feedback provided to improve the rationale and answer.")
    continue_check: bool = Field(description="Indicates whether the answer of current reasoning is the same as before as True (if needed) or False (if not).")

import re

from tqdm import tqdm


def extract_json(text):
    # Find the first opening { and last closing } bracket
    start = text.find('{')
    end = text.rfind('}')
    
    if start == -1 or end == -1:
        return text
        
    # Extract the JSON string
    json_str = text[start:end + 1]
    
    return json_str
    
# Set up a parser + inject instructions into the prompt template.
refine_parser = JsonOutputParser(pydantic_object=RefinerParser)
feedback_parser = JsonOutputParser(pydantic_object=FeedbackParser)

feedback_prompt = ChatPromptTemplate(
    messages=feedback_message,
    partial_variables={"format_instructions": feedback_parser.get_format_instructions()},
)
tokens_total = 0
prompt_and_model = feedback_prompt | llm_llama
output_openai  = prompt_and_model.invoke(arguments_dict)
tokens = output_openai.response_metadata['token_usage']['total_tokens']
result = feedback_parser.invoke(output_openai)
feedback = result['feedback']
cont = result['continue_check']
tokens_total += tokens

In [51]:



# Define function for processing a single row
def process_row(row, row_index):
    print(f"Processing row {row_index}...")  # Inform which row is being processed

    # Extract necessary fields
    question = row['Question']
    rationale = row['reasoning']
    prev_answer = row['parsed_answer']

    arguments_dict = {
        'question': question,
        'rationale': rationale
    }

    # Initialize parsers and tokens
    tokens_total = 0
    previous_answer = prev_answer  # Track if the answer changes

    # Generate initial feedback
    feedback_prompt = ChatPromptTemplate(
        messages=feedback_message,
        partial_variables={"format_instructions": feedback_parser.get_format_instructions()},
    )
    prompt_and_model = feedback_prompt | llm_llama
    output_openai = prompt_and_model.invoke(arguments_dict)

    # Extract feedback and tokens
    tokens = output_openai.response_metadata['token_usage']['total_tokens']
    count = 0
    try:
        result = feedback_parser.invoke(output_openai)
        feedback = result['feedback']
        cont = result['continue_check']
    except Exception as e:
        print(f"Error parsing feedback for row {row_index}: {e}")
        feedback = "Unable to Generate feedback. Please check the original rationale."
        cont = False
        print(f"Default feedback used for row {row_index}: {feedback}")
    tokens_total += tokens

    # Start refinement process
    max_iterations = 10
    current_iteration = 0
    revised_answer = prev_answer  # Initialize with previous answer
    is_answer_different = False
    if isinstance(cont, str):
        cont = cont.strip().lower() == "true"
    while (current_iteration < max_iterations) and cont:
        print(cont)
        current_iteration += 1
        
        print(f"Row {row_index}, Iteration {current_iteration}: Refining answer...")  # Inform about iterations

        # Create refinement prompt
        refine_prompt = ChatPromptTemplate(
            messages=refiner_message,
            partial_variables={"format_instructions": refine_parser.get_format_instructions()},
        )
        refine_prompt_and_model = refine_prompt | llm_llama
        refine_output = refine_prompt_and_model.invoke({
            "question": question,
            "rationale": rationale,
            "feedback": feedback
        })

        # Extract tokens and add to total
        refine_tokens = refine_output.response_metadata['token_usage']['total_tokens']
        tokens_total += refine_tokens

        # Parse refined result
        try:
            output_text = extract_json(refine_output.content)
            refine_result = refine_parser.invoke(output_text)
            revised_answer = refine_result['answer']
            updated_reasoning = refine_result['revised_rationale']
        except Exception as e:
            print(f"Error parsing refinement result for row {row_index}, iteration {current_iteration}: {e}")
            revised_answer = prev_answer
            updated_reasoning = rationale
            continue_check = False
            print(f"Default values used for refinement result in row {row_index}, iteration {current_iteration}.")

        # Check if answer is different
        if revised_answer != prev_answer:
            is_answer_different = True

        # Update rationale for the next iteration
        rationale = updated_reasoning
        # Stop if no further refinement is needed
        print(f"Row {row_index}, Iteration {current_iteration}: Generating new feedback...")

        feedback_prompt = ChatPromptTemplate(
            messages=feedback_message,
            partial_variables={"format_instructions": feedback_parser.get_format_instructions()},
        )
        prompt_and_model = feedback_prompt | llm_llama
        output_openai = prompt_and_model.invoke({"question": question, "rationale": rationale})

        # Extract feedback and tokens
        tokens = output_openai.response_metadata['token_usage']['total_tokens']
        try:
            result = feedback_parser.invoke(output_openai)
            feedback = result['feedback']
            cont = result['continue_check']
        except Exception as e:
            print(f"Error parsing feedback for row {row_index}: {e}")
            feedback = "Unable to Generate feedback. Please check the original rationale."
            cont = False
            print(f"Default feedback used for row {row_index}: {feedback}")
        tokens_total += tokens

    print(f"Row {row_index}: Completed processing.")  # Inform when processing of a row is complete
    return revised_answer, tokens_total, is_answer_different


# Apply the function to the first 100 rows with tqdm progress bar
results = []
for index, row in tqdm(df.iloc[5000:].iterrows(), total=len(df) - 5000, desc="Processing Rows (Excluding Top 500)"):
    revised_answer, tokens_total, is_answer_different = process_row(row, index)
    results.append((revised_answer, tokens_total, is_answer_different))

# Create new columns in the DataFrame
df.loc[5000:, 'revised_answer'] = [res[0] for res in results]
df.loc[5000:, 'total_tokens'] = [res[1] for res in results]
df.loc[5000:, 'Is Answer Different'] = [res[2] for res in results]







Processing Rows (Excluding Top 500):   0%|                                                                                                                                                                                          | 0/5890 [00:00<?, ?it/s]

Processing row 5000...


Processing Rows (Excluding Top 500):   0%|                                                                                                                                                                                | 1/5890 [00:02<4:47:49,  2.93s/it]

Row 5000: Completed processing.
Processing row 5001...


Processing Rows (Excluding Top 500):   0%|                                                                                                                                                                                | 2/5890 [00:05<4:54:03,  3.00s/it]

...

Row 5190: Completed processing.
Processing row 5191...


Processing Rows (Excluding Top 500):   3%|█████▌                                                                                                                                                                      | 192/5890 [12:45<100:11:41, 63.30s/it]

Row 5191: Completed processing.
Processing row 5192...


Processing Rows (Excluding Top 500):   3%|█████▋                                                                                                                                                                        | 192/5890 [17:46<8:47:41,  5.56s/it]


InternalServerError: <!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>


<title>api.lambdalabs.com | 524: A timeout occurred</title>
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta name="robots" content="noindex, nofollow" />
<meta name="viewport" content="width=device-width,initial-scale=1" />
<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/main.css" />


</head>
<body>
<div id="cf-wrapper">
    <div id="cf-error-details" class="p-0">
        <header class="mx-auto pt-10 lg:pt-6 lg:px-8 w-240 lg:w-full mb-8">
            <h1 class="inline-block sm:block sm:mb-2 font-light text-60 lg:text-4xl tex

In [54]:
df.loc[5000:, 'revised_answer'] = [res[0] for res in results]
df.loc[5000:, 'total_tokens'] = [res[1] for res in results]
df.loc[5000:, 'Is Answer Different'] = [res[2] for res in results]

ValueError: Must have equal len keys and value when setting with an iterable

In [57]:
df.iloc[5100:5200]

,question_id,category,name,question_text,correct_answer,model,repeat_number,answer,reasoning,parsed_answer,is_correct,fail_reason,Question,Type,revised_answer
5100,415,control room,CommonsenseQA_test,Where is a control room needed to prevent wide...,D,llama3.1-70b-instruct-berkeley,0,D) nuclear plant,"A control room is crucial at a nuclear plant, ...",D,True,'parsed_number',Where is a control room needed to prevent wide...,Commonsense,nan
5101,415,control room,CommonsenseQA_test,Where is a control room needed to prevent wide...,D,llama3.1-70b-instruct-berkeley,1,D) nuclear plant,A control room is essential in a nuclear plant...,D,True,'parsed_number',Where is a control room needed to prevent wide...,Commonsense,nan
5102,415,control room,CommonsenseQA_test,Where is a control room needed to prevent wide...,D,llama3.1-70b-instruct-berkeley,2,D) nuclear plant,A control room is necessary at a nuclear plant...,D,True,'parsed_number',Where is a control room needed to prevent wide...,Commonsense,nan
5103,415,control room,CommonsenseQA_test,Where is a control room needed to prevent wide...,D,llama3.1-70b-instruct-berkeley,3,D) nuclear plant,A control room is essential in a nuclear plant...,D,True,'parsed_number',Where is a control room needed to prevent wide...,Commonsense,nan
5104,415,control room,CommonsenseQA_test,Where is a control room needed to prevent wide...,D,llama3.1-70b-instruct-berkeley,4,D) nuclear plant,A control room is crucial in a nuclear plant t...,D,True,'parsed_number',Where is a control room needed to prevent wide...,Commonsense,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5195,421,fragile,CommonsenseQA_test,"Though the thin film seemed fragile, for it's ...",D,llama3.1-70b-instruct-berkeley,5,B) durable,The key phrase in the question is 'for its int...,B,False,'parsed_number',"Though the thin film seemed fragile, for it's ...",Commonsense,nan
5196,421,fragile,CommonsenseQA_test,"Though the thin film seemed fragile, for it's ...",D,llama3.1-70b-instruct-berkeley,6,A) indestructible,The word 'indestructible' best fits the contex...,A,False,'parsed_number',"Though the thin film seemed fragile, for it's ...",Commonsense,nan
5197,421,fragile,CommonsenseQA_test,"Though the thin film seemed fragile, for it's ...",D,llama3.1-70b-instruct-berkeley,7,A) indestructible,The thin film seemed fragile but was actually ...,A,False,'parsed_number',"Though the thin film seemed fragile, for it's ...",Commonsense,nan
5198,421,fragile,CommonsenseQA_test,"Though the thin film seemed fragile, for it's ...",D,llama3.1-70b-instruct-berkeley,8,E) unbreakable,Given that the thin film seemed fragile but se...,E,False,'parsed_number',"Though the thin film seemed fragile, for it's ...",Commonsense,nan


In [53]:
results

[('B', 499, False),
 ('B', 517, False),
 ('B', 506, False),
 ('B', 481, False),
 ('B', 578, False),
 ('B', 534, False),
 ('B', 535, False),
 ('B', 521, False),
 ('B', 521, False),
 ('B', 504, False),
 ('D', 557, False),
 ('D', 480, False),
 ('D', 516, False),
 ('D', 531, False),
 ('D', 501, False),
 ('D', 525, False),
 ('A', 573, False),
 ('A', 498, False),
 ('D', 520, False),
 ('D', 548, False),
 ('D', 554, False),
 ('D', 547, False),
 ('D', 575, False),
 ('D', 494, False),
 ('B', 485, False),
 ('C', 532, False),
 ('C', 477, False),
 ('C', 536, False),
 ('C', 543, False),
 ('C', 512, False),
 ('C', 557, False),
 ('C', 555, False),
 ('C', 520, False),
 ('C', 529, False),
 ('C', 551, False),
 ('C', 532, False),
 ('C', 532, False),
 ('C', 508, False),
 ('C', 550, False),
 ('C', 462, False),
 ('A) space', 2023, True),
 ('A', 497, False),
 ('A', 569, False),
 ('B', 565, False),
 ('E', 494, False),
 ('A', 577, False),
 ('A', 531, False),
 ('A', 469, False),
 ('A', 588, False),
 ('A', 574, F

In [52]:
result

{'feedback': "The rationale provided is mostly correct but can be improved by explicitly stating that 2001 is not a leap year and that Jane's birthday is on the last day of February, which is the 28th in non-leap years. Additionally, the rationale could benefit from mentioning why the other options are incorrect, specifically why options (A), (B), (D), (E), and (F) are not the correct dates one year ago from today.",
 'continue_check': 'false'}

In [ ]:
# Save the updated DataFrame
df.to_csv('updated_results_llama.csv', index=False)
print("Processing complete. Results saved to 'updated_results_excluding_top_500.csv'.")


In [98]:
output_openai

AIMessage(content='{\n  "feedback": "The rationale states that Jane is celebrating her 16th birthday on February 28, 2017, which is incorrect. If she was born on February 28, 2001, her 16th birthday would actually be on February 28, 2017. Therefore, one year ago from her 16th birthday (February 28, 2017) is indeed February 28, 2016, which is correct. However, the options provided do not include this date in the correct MM/DD/YYYY format (02/28/2016 is missing from the options). To improve, ensure that the options provided include the correct answer based on the rationale given. Additionally, clarify the calculation timeline in the rationale to avoid confusion regarding the birthday and the date one year ago." \n}', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 167, 'prompt_tokens': 365, 'total_tokens': 532, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens':

In [ ]:

# Save the updated DataFrame
df.to_csv('updated_results.csv', index=False)
print("Processing complete. Results saved to 'updated_results.csv'.")

In [ ]:
# Save the updated DataFrame
df.to_csv('updated_results.csv', index=False)

In [20]:
# import re
# def extract_json(text):
#     pattern = r'{.*?}'
#     match = re.search(pattern, text)
#     return match.group() if match else text

{'Answer': '88', 'Confidence': 0.9}

In [22]:
output_text

'{\n  "Answer": "88",\n  "Confidence": 0.9\n}'

In [23]:
## cases

# o find the product of the given polynomials in the given polynomial ring, we need to multiply f(x) and g(x) and then take the result modulo 8.                                                                                 | 3/5 [01:00<00:40, 20.30s/it]

# f(x) = 4x - 5
# g(x) = 2x^2 - 4x + 2

# First, we multiply f(x) and g(x):

# (4x - 5)(2x^2 - 4x + 2) = 
# 8x^3 - 16x^2 + 8x - 10x^2 + 20x - 10 = 
# 8x^3 - 26x^2 + 28x - 10

# Now, we take the result modulo 8:

# 8x^3 (mod 8) = 0
# -26x^2 (mod 8) = 2x^2
# 28x (mod 8) = 4x
# -10 (mod 8) = 6

# So, the result is: 2x^2 + 4x + 6

# However, we are given a choice of answers that seems to not include our exact answer, but option B seems closest to our answer.


# Answer: B) 6x^2 + 4x + 6 
# Confidence: 0.9
# 111
# To find the product of the given polynomials in the given polynomial ring, we need to multiply f(x) and g(x) and then take the result modulo 8.

# f(x) = 4x - 5
# g(x) = 2x^2 - 4x + 2

# First, we multiply f(x) and g(x):

# (4x - 5)(2x^2 - 4x + 2) = 
# 8x^3 - 16x^2 + 8x - 10x^2 + 20x - 10 = 
# 8x^3 - 26x^2 + 28x - 10

# Now, we take the result modulo 8:

# 8x^3 (mod 8) = 0
# -26x^2 (mod 8) = 2x^2
# 28x (mod 8) = 4x
# -10 (mod 8) = 6

# So, the result is: 2x^2 + 4x + 6

# However, we are given a choice of answers that seems to not include our exact answer, but option B seems closest to our answer.


# Answer: B) 6x^2 + 4x + 6 
# Confidence: 0.9

##

In [24]:
prompt_and_model = prompt | llm_claude
output_claude = prompt_and_model.invoke({"question": question})
output_text = extract_json(output_claude.content)
parser.invoke(output_text)

OutputParserException: Invalid json output: Let me solve this step by step:
1. Total eggs laid per day: 16 eggs
2. Eggs used for breakfast: 3 eggs
3. Eggs used for muffins: 4 eggs
4. Remaining eggs for selling: 16 - 3 - 4 = 9 eggs
5. Price per egg at farmers' market: $2
6. Daily earnings from selling eggs: 9 eggs × $2 = $18

{
    "Answer": "18",
    "Confidence": "0.95"
}
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE

In [25]:
output_text

'Let me solve this step by step:\n1. Total eggs laid per day: 16 eggs\n2. Eggs used for breakfast: 3 eggs\n3. Eggs used for muffins: 4 eggs\n4. Remaining eggs for selling: 16 - 3 - 4 = 9 eggs\n5. Price per egg at farmers\' market: $2\n6. Daily earnings from selling eggs: 9 eggs × $2 = $18\n\n{\n    "Answer": "18",\n    "Confidence": "0.95"\n}'

In [26]:
output_claude.content

'Let me solve this step by step:\n1. Total eggs laid per day: 16 eggs\n2. Eggs used for breakfast: 3 eggs\n3. Eggs used for muffins: 4 eggs\n4. Remaining eggs for selling: 16 - 3 - 4 = 9 eggs\n5. Price per egg at farmers\' market: $2\n6. Daily earnings from selling eggs: 9 eggs × $2 = $18\n\n{\n    "Answer": "18",\n    "Confidence": "0.95"\n}'

In [56]:
prompt_and_model = prompt | llm_gemini
output = prompt_and_model.invoke({"question": question})
output_text = extract_json(output.content)
parser.invoke(output_text)

{'Answer': '18', 'Confidence': '1'}

In [58]:
prompt_and_model = prompt | llm_azure
output = prompt_and_model.invoke({"question": question})
output_text = extract_json(output.content)
parser.invoke(output_text)

{'Answer': '18', 'Confidence': '1'}

In [69]:
prompt_and_model = prompt | llm_llama
output = prompt_and_model.invoke({"question": question})
output_text = extract_json(output.content)
parser.invoke(output_text)

{'Answer': '18', 'Confidence': '1'}